In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'PyRegridding/Regridder/')

# Own local packages
import AveragingUtils as Av
import VertRegridFlexLL as Vrg
import PlotUtil as Pu

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( Pu )
importlib.reload(Av)

In [ ]:
###################################################
# Add time coordinate to monthly native ERA5 files
###################################################
day=1
for mon in np.arange( start=1 , stop=13 ):
    monA=str(mon).zfill(2)
    for year in np.arange( start=1980 , stop=2023 ):   #, stop=2024 ):
        yearA = str( year )
        path_ERA = '/glade/campaign/cgd/amp/juliob/ERA5/native_monthly/ERA5.native.'+yearA+'-'+monA+'.nc'
        timevar = cftime.DatetimeGregorian(year, mon, day) #(year-1979)*12 + mon-1
        #print(f" {path_ERA} TIME = {timevar} ")
        path_ERA_out = '/glade/campaign/cgd/amp/juliob/ERA5/native_monthly/ERA5.native.time.'+yearA+'-'+monA+'.nc'
        X = xr.open_mfdataset( path_ERA )
        X = X.assign_coords(time=('time', [timevar]))
        print(X['time'])
        X.to_netcdf( path_ERA_out )

In [ ]:
###################################################
# Generate multi year monthly native ERA5 files
###################################################
day=1
for mon in np.arange( start=1 , stop=13 ):
    monA=str(mon).zfill(2)
    yearA = "*"
    multiyearA="1979_2022"
    path_ERA = '/glade/campaign/cgd/amp/juliob/ERA5/native_monthly/ERA5.native.time.'+yearA+'-'+monA+'.nc'
    path_ERA_out = '/glade/campaign/cgd/amp/juliob/ERA5/native_monthly_climo/ERA5.native.time.'+multiyearA+'-'+monA+'.nc'
    X = xr.open_mfdataset( path_ERA )
    XX = X.mean(dim='time')
    XX = XX.expand_dims({'time': 1})
    XX = XX.assign_coords(time=('time', [mon]))
    #XX = XX.expand_dims({'nbnd': 2})

    ################
    # Make time_bnds variable
    ################
    # Extracting the cftime objects from the DataArray
    time0_cftime = X.time.values[0]  # This should now be a cftime.DatetimeGregorian object
    time1_cftime = X.time.values[-1]  # Ditto
    # Convert cftime.DatetimeGregorian to numpy.datetime64
    time0_np = np.datetime64(time0_cftime) #.isoformat())
    time1_np = np.datetime64(time1_cftime) #.isoformat())
    time_bnds =  np.array ( [time0_np,time1_np] )
    Time_Bnds = xr.DataArray(data=time_bnds,dims=['nbnd'],coords=dict( nbnd=[0,1],) )
    XX['time_bnds']=Time_Bnds

    print(f"writing {path_ERA_out} ")
    XX.to_netcdf( path_ERA_out )

    

In [ ]:
XX


In [ ]:
time_bnds = XX.time_bnds
print(time_bnds[0].values[0])

In [ ]:
(XX['time_bnds'].dtype == 'datetime64[ns]' )

In [ ]:
######
# Testing out files

f='/glade/campaign/cgd/amp/juliob/ERA5/native_monthly_climo/ERA5.native.time.1979_2022-*.nc'

In [ ]:
X=xr.open_mfdataset( f  ,data_vars='different', coords='different' )

In [ ]:


X


In [ ]:
years = X['time_bnds'].values[0].astype('datetime64[Y]' ).astype(int)
months = X['time_bnds'].values[0].astype('datetime64[M]' ).astype(int) %12 +1
print(np.shape(months))
print(months)
print(years)


In [ ]:
for k in np.arange(137):
    print( f" {100_000.*X.hyam.values[0,k] +100_000.*X.hybm.values[0,k] :.2f}" ) 

In [ ]:
print(X['time_bnds'].values)
time_array = X['time_bnds'].values[:,0]
print(np.shape( time_array))
#print(time_array[0,:])

In [ ]:
time_array = X['time_bnds'].values[:,0]
months=[]
years=[]
for ixtime in time_array:
    years.append( ixtime.astype('datetime64[Y]' ).astype(int) + 1970 )           
    months.append( ixtime.astype('datetime64[M]' ).astype(int) %12 +1 )                


In [ ]:
print(years)

In [ ]:
time_array[2,0].astype('datetime64[M]' ).astype(int) %12 +1 

In [ ]:
# Convert to month resolution
month_res = time_array.astype('M8[M]')

# Extract years and months
years = np.array([[date.year for date in row] for row in month_res])
months = np.array([[date.month for date in row] for row in month_res])